# 🌿 UrbanEcoSoundMonitor
## Акустический мониторинг городской экосистемы

Проектный практикум (группа: MIFIML-2, семестр 1).

**Идея продукта:**
автоматически анализировать аудиозаписи из городской среды, подсчитывать число событий
разных типов и формировать индикаторы шумового загрязнения и потенциально опасных
ситуаций (частые сирены, выстрелы, строительный шум).

Этот ноутбук демонстрирует работу модели для экологического мониторинга звуков в городской среде.
Используется модель **MIT/ast-finetuned-urban8k**, обученная на UrbanSound8K,
что обеспечивает хорошую классификацию уличных звуков.

**Пайплайн обработки:**

1. Загрузка аудио-файлов (.wav)
2. Предобработка и нормализация
3. Классификация коротких сегментов
4. Построение статистики распределения классов
5. Визуализация результатов

# 1. Импорты и подготовка модели

In [ ]:
from pathlib import Path
from typing import List, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchaudio
from transformers import pipeline as hf_pipeline

# Настройки отображения pandas
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 10)
pd.set_option("display.width", 120)

%matplotlib inline

# Имя модели на Hugging Face
MODEL_NAME = "xpariz10/ast-finetuned-audioset-10-10-0.4593-finetuning-ESC-50"

# Глобальный аудио-пайплайн (как в streamlit_app.py)
audio_pipe = hf_pipeline(
    task="audio-classification",
    model=MODEL_NAME,
    # -1 = CPU
    device=-1,
)

# 2. Функция загрузки аудио

In [ ]:
def load_audio(path: Path, target_sr: int = 16000) -> tuple[np.ndarray, int]:
    """
    Загружает аудио и приводит его к нужному sample rate.

    Возвращает:
        waveform_np: np.ndarray формы (num_samples,)
        sr: частота дискретизации после возможного ресемплинга
    """
    waveform, sr = torchaudio.load(str(path))

    # стерео -> моно
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Ресемплинг (преобразование)
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(sr, target_sr)
        waveform = resampler(waveform)
        sr = target_sr

    return waveform.squeeze(0).numpy().astype(np.float32), sr

# 3. Классификация одного файла

In [ ]:
def classify_audio_segment(audio_np: np.ndarray, sr: int = 16000) -> tuple[str, float]:
    """
    Классифицирует аудио с помощью глобального audio_pipe.

    Возвращает:
        label: название класса
        score: вероятность (0..1)
    """
    result = audio_pipe(
        {"array": audio_np, "sampling_rate": sr},
        top_k=1,
    )[0]
    return result["label"], float(result["score"])

# 4. Пакетная обработка всех файлов

In [ ]:
def process_folder(folder: Path, top_k: int = 1) -> pd.DataFrame:
    """
    Обрабатывает все .wav-файлы в папке.

    Возвращает DataFrame с колонками:
        filename, label, score
    """
    records = []

    wav_files = sorted(folder.glob("*.wav"))
    for file in wav_files:
        audio_np, sr = load_audio(file)
        label, score = classify_audio_segment(audio_np, sr)
        records.append({"filename": file.name, "label": label, "score": score})

    return pd.DataFrame(records)

# 5. Анализ статистики

In [ ]:
def plot_statistics(df: pd.DataFrame) -> pd.Series:
    """
    Строит bar-chart распределения предсказанных top-классов.

    Возвращает:
        Series с количеством файлов по каждому классу.
    """
    if df.empty:
        raise ValueError("Пустой DataFrame, нечего визуализировать")

    counts = df["top_label"].value_counts().sort_values(ascending=False)

    plt.figure(figsize=(10, 6))
    counts.plot(kind="bar")
    plt.title("Распределение обнаруженных звуков (по top-классу)")
    plt.xlabel("Класс")
    plt.ylabel("Количество файлов")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    return counts

# 6. Запуск анализа

In [ ]:
# Путь к папке с WAV-файлами (относительно корня проекта)
AUDIO_FOLDER = Path("data/audio_samples")

df_results = process_folder(AUDIO_FOLDER)
df_results

# 7. Построение статистики

In [ ]:
stats = plot_statistics(df_results)
stats